# Homework 3: Neural Networks

## Part 1: Data Preprocessing

### Read data

In [1]:
import pandas as pd

file_path = './amazon_reviews_us_Office_Products_v1_00.tsv'
df = pd.read_csv(file_path, delimiter='\t', on_bad_lines='skip')
df.dropna()

/var/folders/4k/8jk4g7fx3xl9p5mzpl0rkqj40000gp/T/ipykernel_85269/3029288783.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, delimiter='\t', on_bad_lines='skip')


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,43081963,R18RVCKGH1SSI9,B001BM2MAC,307809868,"Scotch Cushion Wrap 7961, 12 Inches x 100 Feet",Office Products,5,0.0,0.0,N,Y,Five Stars,Great product.,2015-08-31
1,US,10951564,R3L4L6LW1PUOFY,B00DZYEXPQ,75004341,"Dust-Off Compressed Gas Duster, Pack of 4",Office Products,5,0.0,1.0,N,Y,"Phffffffft, Phfffffft. Lots of air, and it's C...",What's to say about this commodity item except...,2015-08-31
2,US,21143145,R2J8AWXWTDX2TF,B00RTMUHDW,529689027,Amram Tagger Standard Tag Attaching Tagging Gu...,Office Products,5,0.0,0.0,N,Y,but I am sure I will like it.,"Haven't used yet, but I am sure I will like it.",2015-08-31
3,US,52782374,R1PR37BR7G3M6A,B00D7H8XB6,868449945,AmazonBasics 12-Sheet High-Security Micro-Cut ...,Office Products,1,2.0,3.0,N,Y,and the shredder was dirty and the bin was par...,Although this was labeled as &#34;new&#34; the...,2015-08-31
4,US,24045652,R3BDDDZMZBZDPU,B001XCWP34,33521401,"Derwent Colored Pencils, Inktense Ink Pencils,...",Office Products,4,0.0,0.0,N,Y,Four Stars,Gorgeous colors and easy to use,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2640249,US,53005790,RLI7EI10S7SN0,B00000DM9M,223408988,PalmOne III Leather Belt Clip Case,Office Products,4,26.0,26.0,N,N,Great value! A must if you hate to carry thing...,I can't live anymore whithout my Palm III. But...,1998-12-07
2640250,US,52188548,R1F3SRK9MHE6A3,B00000DM9M,223408988,PalmOne III Leather Belt Clip Case,Office Products,4,18.0,18.0,N,N,Attaches the Palm Pilot like an appendage,Although the Palm Pilot is thin and compact it...,1998-11-30
2640251,US,52090046,R23V0C4NRJL8EM,0807865001,307284585,Gods and Heroes of Ancient Greece,Office Products,4,9.0,16.0,N,N,"Excellent information, pictures and stories, I...",This book had a lot of great content without b...,1998-10-15
2640252,US,52503173,R13ZAE1ATEUC1T,1572313188,870359649,Microsoft EXCEL 97/ Visual Basic Step-by-Step ...,Office Products,5,0.0,0.0,N,N,class text,I am teaching a course in Excel and am using t...,1998-08-22


### Form classes and sample

In [2]:
cols = ['review_body', 'star_rating']
df = df[cols]

df['review_body'] = df['review_body'].apply(lambda review: '' if pd.isna(review) or isinstance(review, float) else review)

class_1 = df[df['star_rating'].isin([1, 2, 3])]
class_2 = df[df['star_rating'].isin([4, 5])]

sample_size = 50_000
class_1 = class_1.sample(sample_size, random_state=42)
class_2 = class_2.sample(sample_size, random_state=42)

balanced_df = pd.concat([class_1, class_2])
balanced_df.loc[:, 'class'] = balanced_df['star_rating'].apply(lambda x: 1 if x in [1, 2, 3] else 2 if x in [4, 5] else 0)

## Part 2: Word Embedding

### Download gensim Google Word2Vec model

In [3]:
import gensim.downloader as downloader

wv = downloader.load('word2vec-google-news-300')

### Define a basic test

In [4]:
def basic_test(model):
    examples = [
        ('king', 'man', 'woman'),
        ('france', 'paris', 'berlin'),
        ('coffee', 'beans', 'leaves'),
    ]

    for (word1, word2, word3) in examples:
        try:
            result = model.most_similar(positive=[word1, word3], negative=[word2], topn=1)
            print(f'{word1} - {word2} + {word3} = {result[0][0]}')
        except KeyError:
            print(f'"{word1}", "{word2}", or "{word3}" does not exist in this model')

basic_test(wv)

king - man + woman = queen
france - paris + berlin = germany
coffee - beans + leaves = leaving


### Corpus for training model

In [5]:
from gensim import utils

class MyCorpus:
    """An iterator that processes each line in dataset."""

    def __iter__(self):
        data = balanced_df['review_body']
        for line in data:
            yield utils.simple_preprocess(line)

### Train custom model

In [6]:
import gensim.models

sentences = MyCorpus()
model = gensim.models.Word2Vec(sentences=sentences, vector_size=300, window=13, min_count=9, workers=4)
custom_model = model.wv

### Test custom model

In [7]:
basic_test(custom_model)

king - man + woman = openers
"france", "paris", or "berlin" does not exist in this model
coffee - beans + leaves = surface


### Custom vs pretrained model analysis

We conclude that my model is much less precise than the pretrained Google model. For example, king - man + woman = oversize, which has no semantic similarity at all.

## Part 3: Simple Models

### Train test split

In [39]:
from sklearn.model_selection import train_test_split

X = balanced_df['review_body']
y = balanced_df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

### Define average Word2Vec

In [92]:
import numpy as np

def get_average_w2v(review_string, model, k=300):
    review = review_string.split(' ')
    if (len(review) < 1):
        return np.zeros(k)
    
    embeddings = np.zeros(k)
    for word in review:
        try:
            embeddings += model[word]
        except KeyError:
            pass
    
    average_embedding = np.divide(embeddings, len(embeddings))

    return average_embedding

### Create test set using Google model

In [41]:
google_model = wv

X_train_w2v = X_train.apply(lambda review: get_average_w2v(review, google_model))
X_test_w2v = X_test.apply(lambda review: get_average_w2v(review, google_model))

X_train_w2v = np.array(X_train_w2v.tolist())
X_test_w2v = np.array(X_test_w2v.tolist())

### Create test set using TF-IDF vectorizer

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

### Test Perceptron

In [12]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score

perc_w2v = Perceptron()
perc_w2v.fit(X_train_w2v, y_train)
y_pred_perc_w2v = perc_w2v.predict(X_test_w2v)
accuracy_perc_w2v = accuracy_score(y_test, y_pred_perc_w2v)

perc_tfidf = Perceptron()
perc_tfidf.fit(X_train_tfidf, y_train)
y_pred_perc_tfidf = perc_tfidf.predict(X_test_tfidf)
accuracy_perc_tfidf = accuracy_score(y_test, y_pred_perc_tfidf)

print(f'Perceptron accuracy with Word2Vec: {accuracy_perc_w2v}')
print(f'Perceptron accuracy with TF-IDF: {accuracy_perc_tfidf}')

Perceptron accuracy with Word2Vec: 0.7965
Perceptron accuracy with TF-IDF: 0.82425


### Test SVM

In [13]:
from sklearn.svm import LinearSVC

svm_w2v = LinearSVC(dual='auto')
svm_w2v.fit(X_train_w2v, y_train)
y_pred_svm_w2v = svm_w2v.predict(X_test_w2v)
accuracy_svm_w2v = accuracy_score(y_test, y_pred_svm_w2v)

svm_tfidf = LinearSVC(dual='auto')
svm_tfidf.fit(X_train_tfidf, y_train)
y_pred_svm_tfidf = svm_tfidf.predict(X_test_tfidf)
accuracy_svm_tfidf = accuracy_score(y_test, y_pred_svm_tfidf)

print(f'SVM accuracy with Word2Vec: {accuracy_svm_w2v}')
print(f'SVM accuracy with TF-IDF: {accuracy_svm_tfidf}')

SVM accuracy with Word2Vec: 0.78975
SVM accuracy with TF-IDF: 0.8649


## Part 4: Feedforward Neural Networks

### Create Neural Network
Our entry point will be the function `evaluate_features`

In [71]:
import torch
import torch.nn as nn
import torch.optim as optim

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(300, 50),
            nn.ReLU(),
            nn.Linear(50, 5),
            nn.ReLU(),
            nn.Linear(5, 2),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.layers(x)

def train_model(model, train_loader):
    lr = 1e-3
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    num_epochs = 10
    for epoch in range(num_epochs):
        for reviews, labels in train_loader:
            outputs = model(reviews)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')
    
    return model

def evaluate_model(model, test_loader, description):
    with torch.no_grad():
        for reviews, labels in test_loader:
            outputs = model(reviews)
            _, pred = torch.max(outputs, 1)
            accuracy = (pred == labels).sum().item() / labels.size(0)

    print(f'Accuracy with {description}: {accuracy * 100:.4f}')

### Average Word2Vec

#### Define helper function to generate loaders

In [83]:
from torch.utils.data import DataLoader, TensorDataset

def get_train_test_splits(get_w2v):
    X_train_local = X_train.apply(lambda review: get_w2v(review, google_model))
    X_test_local = X_test.apply(lambda review: get_w2v(review, google_model))

    X_train_concat = np.array(X_train_local.values.tolist())
    X_test_concat = np.array(X_test_local.values.tolist())

    return X_train_concat, X_test_concat

def create_loader_from_train_test_splits(X_train, X_test):
    X_train_tensor = torch.FloatTensor(X_train)
    y_train_tensor = torch.LongTensor(y_train.values)
    X_test_tensor = torch.FloatTensor(X_test)
    y_test_tensor = torch.LongTensor(y_test.values)

    train_data = TensorDataset(X_train_tensor, y_train_tensor)
    test_data = TensorDataset(X_test_tensor, y_test_tensor)

    train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

    return train_loader, test_loader

def create_loader(get_w2v):
    X_train_concat, X_test_concat = get_train_test_splits(get_w2v)
    train_loader, test_loader = create_loader_from_train_test_splits(X_train_concat, X_test_concat)
    
    return train_loader, test_loader

#### Build train & test split

In [104]:
train_loader_a, test_loader_a = create_loader(get_average_w2v)

#### Train & evaluate `Average Word2Vec`

In [105]:
model_a = train_model(MLP(), train_loader_a)
evaluate_model(model_a, test_loader_a, 'Average Word2Vec')

Epoch 1/10, Loss: 0.4977
Epoch 2/10, Loss: 0.5192
Epoch 3/10, Loss: 0.4365
Epoch 4/10, Loss: 0.5107
Epoch 5/10, Loss: 0.5178
Epoch 6/10, Loss: 0.5297
Epoch 7/10, Loss: 0.4925
Epoch 8/10, Loss: 0.5193
Epoch 9/10, Loss: 0.4826
Epoch 10/10, Loss: 0.5251
Accuracy with Average Word2Vec: 84.3750


### Concatenated Word2Vec

In [48]:
def get_concatenated_w2v(review_string, model, k=300):
    review = review_string.split(' ')
    if (len(review) < 1):
        return np.zeros(k)
    
    embeddings = np.zeros(k)
    for i in range(10):
        try:
            word = review[i]
            embeddings += model[word]
        except KeyError:
            pass
        except IndexError:
            break
    
    average_embedding = np.divide(embeddings, len(embeddings))

    return average_embedding

#### Build train & test split

In [110]:
train_loader_b, test_loader_b = create_loader(get_concatenated_w2v)

#### Evaluate `Concatenated Word2Vec`

In [111]:
model_b = train_model(MLP(), train_loader_b)
evaluate_model(model_b, test_loader_b, 'Concatenated Word2Vec')

Epoch 1/10, Loss: 0.5884
Epoch 2/10, Loss: 0.6318
Epoch 3/10, Loss: 0.6177
Epoch 4/10, Loss: 0.5616
Epoch 5/10, Loss: 0.5781
Epoch 6/10, Loss: 0.5250
Epoch 7/10, Loss: 0.5094
Epoch 8/10, Loss: 0.5775
Epoch 9/10, Loss: 0.5938
Epoch 10/10, Loss: 0.5785
Accuracy with Concatenated Word2Vec: 65.6250


### Average vs Concatenated Word2Vec Analysis

Intuitively, taking only the first 10 vectors would be more lossy than averaging all the vectors in the review, so naturally test data shows that we have a much lower accuracy of about `62% in Concatenated Word2Vec` compared to about `81% in Average Word2Vec`.

We conclude that the difference in precision is too large for us to effectively use `Concatenated Word2Vec`.

## Part 5: Recurrent Neural Networks

In [78]:
class SimpleRNN(nn.Module):
    def __init__(self):
        super(SimpleRNN, self).__init__()
        self.input_dim = 300
        self.hidden_dim = 10
        self.output_dim = 2
        self.rnn = nn.RNN(self.input_dim, self.hidden_dim, batch_first=True)
        self.fc = nn.Linear(self.hidden_dim, self.output_dim)
    
    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_dim)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

def get_2d_w2v(review_string, model, k=300):
    review = review_string.split()
    embeddings = np.zeros((10, k))

    for i in range(10):
        try:
            word = review[i]
            embeddings[i] = model[word]
        except KeyError:
            pass
        except IndexError:
            break
    
    return embeddings

In [112]:
train_loader_rnn, test_loader_rnn = create_loader(get_2d_w2v)

In [113]:
simple_rnn = train_model(SimpleRNN(), train_loader_rnn)
evaluate_model(simple_rnn, test_loader_rnn, 'RNN')

Epoch 1/10, Loss: 0.5107
Epoch 2/10, Loss: 0.6250
Epoch 3/10, Loss: 0.5401
Epoch 4/10, Loss: 0.5747
Epoch 5/10, Loss: 0.4220
Epoch 6/10, Loss: 0.5225
Epoch 7/10, Loss: 0.5083
Epoch 8/10, Loss: 0.4677
Epoch 9/10, Loss: 0.4700
Epoch 10/10, Loss: 0.5705
Accuracy with RNN: 84.3750
